# Wind Power forecasting for the day-ahead energy market - Data Challenge
by Compagnie Nationale du Rhône, ENS Paris & Collège de France

<p align="center"><img src="https://cap.img.pmdstatic.net/fit/http.3A.2F.2Fprd2-bone-image.2Es3-website-eu-west-1.2Eamazonaws.2Ecom.2Fcap.2F2019.2F10.2F04.2Fea495374-9115-4be7-a91a-e9bc5b305b0b.2Ejpeg/768x432/background-color/ffffff/focus-point/992%2C1086/quality/70/dangereuses-pour-la-sante-peu-ecolo-faut-il-en-finir-avec-les-eoliennes-1352031.jpg" width="600"/></p>

Challenge website: https://challengedata.ens.fr/participants/challenges/34/

The objective of this challenge is to **design and train an ML/DL model to predict the hourly electrical production** of six independent wind farms owned by CNR for the day ahead, using multiple Numerical Weather Predictions (NWP) models. This is a **supervised learning problem** based on **multivariate time series**.

This notebook is **fully compatible with Google Colab**, feel free to try it yourself!
https://colab.research.google.com/github/qcha41/wind-power-forecasting-challenge/blob/master/notebook.ipynb

## Overview & Status

- Training of 6 models in parallel with identical architectures (one per wind farm).
- ReLu activation for the last neuron to output a positive (or null) value for the power production.
- Forward chaining cross-validation (on the last 3x10% of the training dataset)

| Steps | Forward chaining<br />cross-validation<br />error (MSE) | Final test error<br />(CAPE) | 
| ----- | ----- | ----- | 
| **Baseline**<br />*(multi-variable linear regression)* | 7.22 | 62.32 |
| **Feature engineering**<br />*(WS,WD)* | 1.71 **(-76.3%)** | 33.43 **(-46.3%)** |
| **Manual shutdown detection** | 1.51 **(-11.6%)** | 33.42 **(-0%)** |
| **RNN**<br />*(GRU cells)* | | 32.29 **(-3.4%)** |

## Notebook setup

First of all, let's import the required libraries and configure the notebook.

In [ ]:
# Google Colab configuration
!git clone https://github.com/qcha41/wind-power-forecasting-challenge.git
!pip install urllib3==1.25.4 folium==0.2.1 boto3 mlflow mpld3 --quiet
%cd wind-power-forecasting-challenge

In [ ]:
# Load and configure libraires
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import core
import mlflow
import importlib

In [ ]:
# [PRIVATE CELL - SKIP IT] Google Colab & Mlflow on AWS custom configuration
core.utilities.configure_mlflow_server()

## Data
### First exploration

In this challenge, we are provided with a **training dataset** and a **test dataset**.

The **training dataset** is composed of different hourly weather forecasts (X) for a period of 8 consecutive months (from May the 1st of 2018 to January the 15th of 2019), together with the associated observed power production in MW (Y). In the **test dataset**, only predictions are provided for another period of 8 months (January the 16th of 2019 to September the 30rd of 2019). The performance of our model is then evaluated online, by submitting its predictions on the test dataset.

In [ ]:
# Load data
df_raw = core.data.load_data()
df_raw.loc[51:54]

A given **training example** is thus composed of :
 - a **target time** (*Time* column) and a **wind farm ID** (*WF* column).
 - several **weather forecasts** for that (*Time*,*WF*) couple, in the form of *runs* (*NWP\<i>_\<HourOfTheRun>_\<DayOfTheRun>_\<Variable>* columns). Each run provides an estimation of a particular weather *Variable*, produced at a given time before the target *Time* (*HourOfTheRun*, *DayOfTheRun*), and coming from a given NWP models (*i*). For instance, the run *NWP1_00h_D-2_U* is estimating the weather variable *U* for a given target *Time* using the first NWP model, and is produced at midnight two days before the target *Time*. 
 - the **observed power production** (*Production* column) for that (*Time*,*WF*) couple.

The runs are coming from 4 different NWP models ($i\in[1,4]$), and are forecasting 4 weather variables at various time:
 
NWP Variable | Prediction description | NWP 1 (hourly) | NWP 2 (every 3 hours) | NWP 3 (every 3 hours) | NWP 4 (hourly)
------ | ----- | ----- | ----- | ----- | -----
Wind speed U,V (m/s) | 10min average [H-10min,H] | x (@100m) | x (@100m) | x (@100m) | x (@10m)
Temperature of air T (m/s) | 1hour average [H-1,H] | x |  | x |
Total cloud cover CLCT (%) | instant value at H | | | | x

Further details about these forecasts can be found on the challenge webpage (link above).

In [ ]:
# Display the number of training examples per wind farm
df_raw.assign(training = df_raw.Production.isna(), test = ~df_raw.Production.isna())[['WF','training','test']].groupby('WF').sum()

### Data reshaping

In order to train a model, we first need to design and shape the training examples that will feed it. In this problem, the learning features are the different weather forecasts, and the target output is the observed power production. 

Here are some characteristics of the NWP forecasts:
 - some NWP models are not forecasting their weather variables hourly.
 - a NWP model is forecasting a given weather variable several times before the target time.

In [ ]:
# Some NWP models are not forecasting a given weather variable hourly
df_raw.loc[51:54,['WF','Time']+[f'NWP{i}_00h_D-2_U' for i in range(1,5)]]

In [ ]:
# For a given target time, a NWP model is forecasting a weather variable several times (at different delays before the target time).
df_raw.loc[51:54,['WF','Time']+[col for col in df_raw.columns if col.startswith('NWP4') and col.endswith('_U')]]

My approach to deal with the induced missing values is to **compute a new matrix containing the best weather forecast for each (*WF*, *NWP*, *Variable*) triplet**. In other words, each of these triplet is reduced to only one value. For instance, I will create a new single feature called *NWP4_U* giving the best forecast for the wind component *U* forecasted by the fourth NWP model, using the different forecasts *NWP4_XXh-XXX_U*.

To do that, I am using a **weighted mean of the forecasts with a memory coefficient <img src="https://render.githubusercontent.com/render/math?math=\alpha"/>** as hyperparameter. This allows to make recent forecasts more predominant in the calculation than the older ones. We have then : 

<img src="https://render.githubusercontent.com/render/math?math=V_{best}=\dfrac{\sum_{k=1}^{n}\alpha^{\Delta H_k}\,V_k}{\sum_{k=1}^{n}\alpha^{\Delta H_k}}"/>

where <img src="https://render.githubusercontent.com/render/math?math=V_k"/> is the k-th prediction made for a given triplet, which has been produced <img src="https://render.githubusercontent.com/render/math?math=\Delta H_k"/> hours before the target time. 
<img src="https://render.githubusercontent.com/render/math?math=\alpha"/> is a memory coefficient lying in <img src="https://render.githubusercontent.com/render/math?math=]0,1]"/>, which make the value weight <img src="https://render.githubusercontent.com/render/math?math=\alpha^{\Delta H_k}"/> decaying as the delay <img src="https://render.githubusercontent.com/render/math?math=\Delta H_k"/> increases. If <img src="https://render.githubusercontent.com/render/math?math=\alpha=1"/>, all predictions have the same weight (classic mean). Instead, if <img src="https://render.githubusercontent.com/render/math?math=\alpha"/> tends towards 0, we are just keeping the more recent forecast. 

Let's start with <img src="https://render.githubusercontent.com/render/math?math=\alpha=0.9"/>, meaning that the (H-1) forecast, if existing, has weight 0.9, then the (H-2) forecast has weight <img src="https://render.githubusercontent.com/render/math?math=(0.9)^2=0.81"/>, the (H-12) forecast has weight <img src="https://render.githubusercontent.com/render/math?math=(0.9)^12=0.28"/>, and so on.

In [ ]:
# Compute best weather forecasts
FORECAST_MEMORY = 0.9
df_best = core.data.calculate_best_forecasts(df_raw, FORECAST_MEMORY)
df_best.loc[51:54]

### Data insights & features engineering

**Data interpolation** - Previously, we have drastically reduced the number of original features by making new ones, more meaningful for training the future ML/DL model. But we are still facing missing values, due to the fact that there are no forecasts at all for these weather variable at these times. A fairly straightforward approach here is thus to **linearly interpolate these missing values using the forecasts made for previous and future times**.

In [ ]:
# Interpolate remaining missing values
df_interp = core.data.interpolate_nans(df_best)
df_interp.loc[51:54]

**Feature engineering** - In order to help the future ML/DL model to understand better the important features of the dataset, we create some of them based on the existing ones:
- Wind speed <img src="https://render.githubusercontent.com/render/math?math=WS=U^2%2BV^2"/>
- Wind direction <img src="https://render.githubusercontent.com/render/math?math=WD=\arctan(U/V)"/>

In [ ]:
# Feature engineering
df_aug = core.data.append_features(df_interp)
df_aug.loc[51:54]

In [ ]:
df_aug = df_interp

**Data insights** - Let's have a quick look on basic dataset statistics.

In [ ]:
# Global overview
df_aug.describe()

Let's now explore a bit more how the features evolves in time. Note the seasonality in the temperature feature. Unfortunately, this seasonality cannot be removed from the dataset because de training period is too short.

In [ ]:
core.plots.feature_vs_time(df_aug,"T") # Try also with T, U, V, CLCT, WS, WD variables

Let's now explore how the power production (target output) evolves in time for each wind farms. We can see that for instance that the maximal power production authorized is reached in WF1 and WF6, and that this limit is different from a wind farm to another (limited at 10 MW for WF1 and at 4 MW for WF6).

In [ ]:
core.plots.production_vs_time(df_aug)

The following correlation graph confirms the high importance of having computed the wind speed *WS*: it is the feature that correlate the most with the power production.

In [ ]:
core.plots.correlation_graph(core.data.mean_data(df_aug))

Furthermore, we can see on the following graph (Production vs WS) that:
 - there is, on average, a linear relationship between the wind speed and power production.
 - the maximal power production capacity is limited differently regarding the wind farms. 
 - WF1 seems to contain a period of forced inactivity (high WS forecasted but 0 MW of power production), which should be excluded from the training set.

In [ ]:
core.plots.production_vs_feature(df_aug,'WS') # Can be tried also with T, U, V, CLCT, WS, WD variables

**Detection and removal of manual shutdown periods** - When exploring the training dataset, we can find some strange long periods of time where the power production of the wind farm is null whereas the wind speed evolves as normal. One example is shown below :

In [ ]:
filter = (df_aug.WF==1)&(df_aug.Production.notna())
df_aug[filter][[col for col in df_aug.columns if col.endswith("WS")]+["Time","Production"]].loc[5200:5750].plot(x="Time")

One reasonable explanation for that is that the wind farm is manually turned off, probably for technical maintenance. Similar cases may appear when the wind speed is too high and that the wind farm is turned off for security reasons. However, such examples are too rare in the current dataset to be correctly learned by the models. Thus, I decided to ignore these problematic periods from the learning dataset when the production of a wind farm is null on a consecutive period of more than 24 hours. 

In [ ]:
def find_inactivity_periods(df):

    inactivity_periods = []
    for wf_num in df.WF.unique():
        df_wf = df[(df.WF==wf_num)&(df.Production.notna())].Production
        inactivity = 0
        for i in range(len(df_wf)) :
            if df_wf.iloc[i] == 0 :
                if inactivity == 0 :
                    ID_ini = df_wf.index.values[i]
                inactivity += 1
            elif inactivity > 0 :
                inactivity_periods.append({'hours':inactivity, 'ID_ini':ID_ini})
                inactivity = 0
    
    return inactivity_periods
    
delay_threshold = 24
inactivity_periods = find_inactivity_periods(df_aug)
core.plots.inactivity_histogram(inactivity_periods, delay_threshold)
df_disabled = core.data.set_disabled_flag(df_aug, inactivity_periods, delay_threshold)
print("Porportion of manual shutdown period detected in training data (True value)")
print(df_disabled[df_disabled.Production.notna()].disabled.value_counts(normalize=True))

In [ ]:
df_disabled = df_aug

### Data normalization
- Standard normalization (z-score) for weather variables *U*, *V*, *T*, *WS*, *WD*.
- Rescaling weather variable *CLCT* percentage between 0 and 1.

The power production is not rescaled (values lying in [0,14]).

In [ ]:
# Data normalization
df = core.data.normalize_data(df_disabled)
df.loc[51:54]

## Neural networks models

The following models are trained using time series mini-batches, of shape (mini-batch size, window size, nb_features).
The window size is the number of previous weather forecasts (corresponding to hours) taken into account to compute the power production at time $t$:
- x_t : weather forecasts from time *t-window_size* to *t* (included), which corresponds to *window_size+1* different weather forecasts.
- y_t : observed power production at time *t*

### Training utilities

In [ ]:
# Training functions
def train(model, t_train, x_train, y_train, t_valid=None, x_valid=None, y_valid=None, epochs=1000, es=True):
    
    validation = True if (x_valid is not None and y_valid is not None) else False
    
    # Make windowed datasets
    dataset_train = tf.data.Dataset.from_tensor_slices((x_train,y_train))
    dataset_train = dataset_train.shuffle(buffer_size=8192, reshuffle_each_iteration=True).batch(32)
    dataset_valid = None
    if validation :
        dataset_valid = tf.data.Dataset.from_tensor_slices((x_valid,y_valid))
        dataset_valid = dataset_valid.batch(256)
        
    # Compile
    model.compile(loss='mse', 
                  optimizer=tf.keras.optimizers.Adam())

    # Fit 
    callbacks = []
    if validation and es : 
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50, min_delta=0.01, baseline=4.5)]
    history = model.fit(dataset_train, 
                          validation_data=dataset_valid,
                          epochs=epochs,
                          callbacks=callbacks,
                          verbose=2)
    core.plots.learning_curves(history)
    
    # Evaluate
    losses = {'loss':model.evaluate(dataset_train)}
    if validation : 
        losses['val_loss'] = model.evaluate(dataset_valid)
        
    return losses


def cross_validation(model_generator, df, wf_num) :

    df_losses = pd.DataFrame()
    data = core.data.WFData(df, wf_num, model_generator.window_size)
    params = {'wf_num':wf_num, 'model_name':model_generator.__class__.__name__}
    
    with mlflow.start_run():
        mlflow.log_params(params)
        
        #for (t_train, x_train, y_train, t_valid, x_valid, y_valid) in data_wf.split_train_valid_holdout(split=0.7) :
        for (t_train, x_train, y_train, t_valid, x_valid, y_valid) in data.split_train_valid_forward_chaining(nb_steps=3, valid_size=0.1) :
            with mlflow.start_run(nested=True):
                mlflow.log_params(params)

                # Train
                model = model_generator.get()
                losses = train(model, t_train, x_train, y_train, t_valid, x_valid, y_valid)
                mlflow.log_metrics(losses)
                df_losses = df_losses.append(losses, ignore_index=True)

                # Predict
                y_train_predict = model.predict(x_train).squeeze()
                y_valid_predict = model.predict(x_valid).squeeze()
                #core.plots.predictions_vs_time(t_train, y_train, y_train_predict, title=f"wf{wf_num}_train_mse={losses['loss']:.2f}")
                core.plots.predictions_vs_time(t_valid, y_valid, y_valid_predict, title=f"wf{wf_num}_valid_mse={losses['val_loss']:.2f}")
        
        mlflow.log_metrics(df_losses.mean().to_dict())
        mlflow.log_metrics(df_losses.std().add_suffix('_std').to_dict())
        df_losses.loc[:,'wf_num'] = wf_num
        
    return df_losses


def full_dataset_training(model_generator, df, wf_num, nested=False):
    
    data = core.data.WFData(df, wf_num, model_generator.window_size)
    params = {'wf_num':wf_num, 'model_name':model_generator.__class__.__name__}
    
    with mlflow.start_run(nested=nested):
        mlflow.log_params(params)
        
        # Train
        t_train, x_train, y_train = data.get_train_data()
        model = model_generator.get()
        losses = train(model, t_train, x_train, y_train, epochs=100, es=False)

        # Predict
        t_test, x_test, y_test = data.get_test_data()
        y_test_predict = pd.Series(model.predict(x_test).squeeze(), index=t_test.index, name="Production")
    
    return y_test_predict

### Multi-variable linear regression (baseline)

Let's start with one simple model to **quickly get a performance benchmark for future more complex models**. We choose the **multivariable linear regression**, which takes as input for a given time only the weather forecasts for that particular time (we are thus not considering time series for now).

In [ ]:
class BaselineModel :
    def __init__(self,df) :
        self.window_size = 1 # hour
        self.nb_features = len(core.data.get_nwp_cols(df))
    def get(self) :
        return tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(self.window_size, self.nb_features)),
            tf.keras.layers.Dense(1, activation='relu')
        ])

See results in the "Overview / Status section".

### Recurrent neural network (RNN)

Now, let's try to catch finer relationships using the fact that training data is made of time series. In the following, we use an RNN model which is trained on 72h of consecutive data. 

In [2]:
class RNNModel :
    def __init__(self, df) :
        self.window_size = 72 # hours
        self.nb_features = len(core.data.get_nwp_cols(df))
    def get(self):
        return tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(self.window_size, self.nb_features)),
            tf.keras.layers.GRU(32, return_sequences=True),
            tf.keras.layers.Dropout(0.6),
            tf.keras.layers.GRU(32, return_sequences=True),
            tf.keras.layers.Dropout(0.6),
            tf.keras.layers.GRU(32),
            tf.keras.layers.Dense(1, activation='relu')
        ])   

See results in the "Overview / Status section".

### Experimentation section

In [ ]:
# EXPRIMENTATION CELL
# ===================

mlflow.set_experiment('trash')
model_generator = RNNModel(df)
#losses = cross_validation(model_generator, df, wf_num=1)
a = full_dataset_training(model_generator, df, wf_num=1)
print(a)

In [ ]:
### Final model evaluation
### ======================

model_generator = RNNModel(df)

# Cross validation for each wind farm
mlflow.set_experiment('final_cv')
df_losses = pd.DataFrame()
for wf_num in df.WF.unique() :
    losses = cross_validation(model_generator, df, wf_num)
    df_losses = pd.concat([df_losses, losses])
gb = df_losses.groupby('wf_num')
perf = pd.concat([gb.mean(),gb.std().add_suffix('_std')],axis=1)

# Full training
mlflow.set_experiment('final_full')
with mlflow.start_run() :
    mlflow.log_params({'wf_num':wf_num, 'model_name':model_generator.__class__.__name__})
    predictions = []
    for wf_num in df.WF.unique() :
        y_test_predict = full_dataset_training(model_generator, df, wf_num, nested=True)
        predictions.append(y_test_predict)
    core.training.save_predictions(predictions)
    core.training.save_data(perf,'performance.csv')
    mlflow.log_metrics(perf.mean().to_dict())